In [ ]:
import pdft
import psi4 

# Neon Example

In [ ]:
ne_geometry = psi4.geometry("""
0 1
Ne
symmetry c1
""")

In [ ]:
#Restricted and Unrestricted calculations

ner = pdft.RMolecule(ne_geometry, "aug-cc-pVQZ", "pbe")
ner.scf()
print("Restricted Energy", ner.energy)
#ner.energetics

neu = pdft.UMolecule(ne_geometry, "cc-pvdz", "pbe")
neu.scf()
print("Unrestricted Energy", ner.energy)
#neu.energetics

In [ ]:
#Plot electron density
neu.axis_plot("z", [neu.Da.np + neu.Db.np], xrange=[0,3], labels=["Helium Electron Density"])

#Plot electron density
#neu.axis_plot("z", [wfn.Da().np], xrange=[0,3], labels=["Helium Electron Density"])

#Differences in electron density
#neu.axis_plot("z", [neu.Da.np - wfn.Da().np], xrange=[0,3], labels=["Helium Electron Density"])

In [ ]:
#Plot Exchange Correlation with pdft
neu.axis_plot("z", [neu.vxc_a.np, neu.vxc_b.np], 
                   xrange=[0,3], yrange=[-5, 0.1], 
                   labels=["vxc_a", "vxc_b"])


#Plot exchange correlation with psi4
# neu.axis_plot("z", [wfn.Va().np, wfn.Vb().np], 
#                    xrange=[0,3], yrange=[-5, 0.1], 
#                    labels=["vxc_a", "vxc_b"])

#Difference between psi4 and pdft
# neu.axis_plot("z", [wfn.Va().np + wfn.Vb().np - (neu.vxc_a.np + neu.vxc_b.np)], 
#                    xrange=[0,3], yrange=[-5, 0.1], 
#                    labels=["vxc_a", "vxc_b"])

# Hydrogen Example | Self Interaction Error

In [ ]:
h_geometry = psi4.geometry("""
0 2
H
symmetry c1
""")

hu = pdft.UMolecule(h_geometry, "aug-cc-pvdz", "svwn")
hu.scf()
print("Hydrogen Energy", hu.energy)

In [ ]:
#Plot electron density
hu.axis_plot("z", [hu.Da.np + hu.Db.np], xrange=[0,3], labels=["Hydrogen Electron Density"])

In [ ]:
#Self Interaction Error Plot

vxc = hu.vxc_a.np + hu.vxc_b.np
hartree = hu.vha_a.np + hu.vha_b.np
sie = vxc + hartree

hu.axis_plot("z", [vxc, hartree, sie], 
                  labels=["Vxc", "VHartree", "SIE"],
                  xrange=[0,3],)

# Inversion Example

In [ ]:
#Be2 

import pdft
import psi4 

#Define Fragments
bea = psi4.geometry("""
0 1 
noreorient
Be 0.0 0.0 0.0
@Be 0.0 0.0 2.3929393
units angstrom
symmetry c1
""")

beb = psi4.geometry("""
0 1 
noreorient
@Be 0.0 0.0 0.0
Be 0.0 0.0 2.3929393
units angstrom
symmetry c1
""")

be2 = psi4.geometry("""
0 1
noreorient
Be 0.0 0.0 0.0
Be 0.0 0.0 2.3929393
units angstrom
symmetry c1
""")


#SCF on fragments

method = "svwn"
basis  = "ugbs"
f_bea = pdft.RMolecule(bea, basis, method)
f_beb = pdft.RMolecule(beb, basis, method)
m_be2 = pdft.RMolecule(be2, basis, method)
f_bea.scf()
f_beb.scf()
m_be2.scf()

#Define inversion object
inv = pdft.Inversion([f_bea, f_beb], m_be2)

In [ ]:
#Perform ZPM method
inv.vp_handler("zpm", beta=5, maxiter=600, print_scf=True, plot_scf=True)

---

In [ ]:
#Dimer

import pdft
import psi4

psi4.set_options({"DFT_SPHERICAL_POINTS" : 110, 
                 "DFT_RADIAL_POINTS" : 70})

w1_geometry = psi4.geometry("""
0 1
O -0.014038 0.177981 0.088324
H 0.239554 -0.111539 0.979257
H 0.774446 0.005137 -0.450548
@O -1.248545 -2.299366 -0.642059
@H -0.973971 -1.386442 -0.428362 
@H -2.139855 -2.198896 -1.005662
symmetry c1
units angstrom
""")
w2_geometry = psi4.geometry("""
0 1
@O -0.014038 0.177981 0.088324
@H 0.239554 -0.111539 0.979257
@H 0.774446 0.005137 -0.450548
O -1.248545 -2.299366 -0.642059
H -0.973971 -1.386442 -0.428362 
H -2.139855 -2.198896 -1.005662
symmetry c1
units angstrom
""")
dimer_geometry = psi4.geometry("""
0 1
O -0.014038 0.177981 0.088324
H 0.239554 -0.111539 0.979257
H 0.774446 0.005137 -0.450548
O -1.248545 -2.299366 -0.642059
H -0.973971 -1.386442 -0.428362 
H -2.139855 -2.198896 -1.005662
symmetry c1
units angstrom
""")

method = "svwn"
basis = "cc-pvdz"

#FRAGMENTS
w1    = pdft.RMolecule(w1_geometry, basis, method)
w2    = pdft.RMolecule(w2_geometry, basis, method)
dimer = pdft.RMolecule(dimer_geometry, basis, method)

w1.scf()
print("w1 energy", w1.energy)
w2.scf()
print("w1 energy", w2.energy)
dimer.scf()
print("dimer energy", dimer.energy)

In [ ]:
winv = pdft.Inversion([w1, w2], dimer)

In [ ]:
winv.vp_handler("zpm", beta=5, maxiter=600, print_scf=True, plot_scf=True)

---